In [1]:
__author__ = 'Mike Fitzpatrick <fitz@noao.edu>'
__version__ = '20200925'
__keywords__ = ['spectra','query','vospace','mydb']

# How to use the Spectrum Data Services

### Table of contents
* [Summary](#summary)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#imports)
* [Review: General template for a simple query in SQL](#review)
* [Quick Example Query](#query)

<a class="anchor" id="summary"></a>
# Summary

This notebook documents how to query, retrieve and visualize spectra from the NOIRLab spectral data service. For full documentation see the <a href="https://datalab.noao.edu/docs/api/specClient.html">API documentation</a>.


### The *specClient* class


The specClient API provides the following methods:

        client = getClient  (context='<context>', profile='<profile>')

          status = isAlive  (svc_url=DEF_SERVICE_URL, timeout=2)

               set_svc_url  (svc_url)
     svc_url = get_svc_url  ()

               set_context  (context)
         ctx = get_context  ()
      ctxs = list_contexts  (optval, token=None, contexts=None, fmt='text')
      ctxs = list_contexts  (token=None, contexts=None, fmt='text')

               set_profile  (profile)
        prof = get_profile  ()
     profs = list_profiles  (optval, token=None, profile=None, fmt='text')
     profs = list_profiles  (token=None, profile=None, fmt='text')

           svcs = services  (name=None, fmt=None, profile='default')

    QUERY INTERFACE:
            id_list = query (<region> | <coord, size> | <ra, dec, size>,
                             constraint=<sql_where_clause>,
                             context='default', profile='default', **kw)

    ACCESS INTERFACE:
            list = getSpec  (id_list, fmt='numpy',
                             out=None, align=False, cutout=None,
                             context='default', profile='default', **kw)

    PLOT  INTERFACE:
                      plot  (spec, context=context, profile=profile, **kw)
         status = prospect  (spec, context=context, profile=profile, **kw)
           image = preview  (id, context=context, profile=profile, **kw)
          image = plotGrid  (id_list, nx, ny, page=<N>,
                             context=context, profile=profile, **kw)
      image = stackedImage  (id_list, fmt='png|numpy',
                             align=False, yflip=False,
                             context=context, profile=profile, **kw)

#### Spectrum Identifiers 

#### Identifier Lists

#### Output Formats
The results can be returned as whitespace delimited (*ascii*), CSV (*csv*), FITS object (*fits*), HDF5 (*hdf5*), or in VOTable format (*votable*). Note that if the results are saved to the user's database then the output format is ignored.

#### Saving Results
If no save location is specified (no *out* param) then the results are returned directly. A save location beginning with the 'vos://' identifier indicates a location in the user's virtual storage to save the result. A save location beginning with the 'mydb://' identifier indicates the results are to be saved to a table in the user's remote database (MyDB). 


### From Python code

The spectrum client service can be called from Python code using the *datalab* module. This provides methods to access the various query client functions in the *specClient* subpackage. See the information [here](https://github.com/noaodatalab/datalab/blob/master/README.md).

Spectrum commands can be also run from the command line, e.g. on your local machine, using the datalab command line utility. Read about it in our GitHub repo [here](https://github.com/noaodatalab/datalab).


<a class="anchor" id="attribution"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noao.edu/disclaimers.php

<a class="anchor" id="imports"></a>
# Imports and setup

This is the setup that is required to use the query client. The first thing to do is import the relevant Python modules.

In [ ]:
# Standard libs
from getpass import getpass               # for password prompts

import os
import numpy as np                        # for manipulating arrays
import pandas as pd
from specutils import Spectrum1D

from io import BytesIO
from astropy import units as u
from astropy.visualization import quantity_support
from astropy.coordinates import SkyCoord

from matplotlib import pyplot as plt      # visualization libs
%matplotlib notebook

# Data Lab imports
#from dl import specClient as spc
import specClient as spc
from dl import storeClient as sc
from dl import queryClient as qc
from dl import authClient as ac
from dl.helpers.utils import convert

In [ ]:
# Global vars for test data
svc_base = "http://gp06.datalab.noao.edu:6999"
svc_base = 'http://localhost:6999'

# Default test single-spectrum
release  = 'dr16'
specobjid = 2210146812474530816
plate, mjd, fiber, run2d = 1963, 54331, 19, 103
bands    = 'flux,loglam,model,ivar'

# Local laptop test data (15,360 files)
# RUN2D = 103, 22 plates/mjd, 640 fibers each
plates = [1960, 1961, 1962, 1963, 2078, 2079, 2174, 2185, 2247, 2255, 2256, 
          2333, 2338, 2377, 2475, 2476, 2667, 2671, 2800, 2821, 2887, 2912]
mjd = [53289, 53299, 53321, 54331, 53378, 53379, 53521, 53532, 53857,
       54169, 53565, 53859, 53682, 53683, 53991, 53845, 53826, 54142,
       54141, 54326, 54393, 54495, 54521, 54499]
plt_mjd = list(zip(plates,mjd))

# Authentication
Much of the functionality of spectrum services can be accessed without explicitly logging into Data (the service then uses an anonymous login). But some capacities, for instance saving the results of your queries to your virtual storage space, require a login (i.e. you will need a registered user account).

If you need to log in to Data Lab, issue this command, and respond according to the instructions:

In [ ]:
#ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

# Query for Spectra

#### 1) FInd spectra in a cone around a specific (RA,Dec) position:

The *query()* method can be called by passing the RA, Dec and a search size (in decimal degrees).

In [ ]:
# RA/Dec query
id_list = spec.query(30.0, 1.0, SIZE, context='sdss',out=''))

#### 2) Find spectra using an Astropy *SkyCoord* object to specify posuition.

In [ ]:
# Astropy SkyCoord objects can be created in a number of ways:
posn = [ SkyCoord(ra=30.0*u.degree, dec=1.0*u.degree, frame='icrs'),   # explicit RA,Dec using units
         SkyCoord(ra=30.0, dec=1.0, frame='icrs', unit='deg'),
         SkyCoord('2h0m0.0s +1d0m0s', frame='icrs')                    # sexagesimal position
       ]

for p in posn:
    print(spec.query(p, SIZE, context='sdss')
    print('-------')

In [ ]:
# Region query
print('######  Region query')
region = [29.95,0.95, 30.05,0.95, 30.05,1.05, 29.95,1.05]
print(spec.query(region, context='sdss',out='', debug=DEBUG))


# #################################
# Constraint tests
# #################################

print('##############  Constraint tests  ##############')
print(spec.query(30.0,1.0,0.05,
                 context='sdss',out='',
                 constraint='z > 0.2 order by z',
                 debug=DEBUG))



# #################################
# Output option tests
# #################################

print('##############  Output tests  ##############')
print('######  Output to stdout')
print(spec.query(30.0,1.0,0.05,
                 out='',
                 context='sdss', debug=DEBUG))

print('######  Output to local file')
fname = '/tmp/test.id'

if os.path.exists(fname): 
    os.remove(fname)

print(spec.query(30.0,1.0,0.05,
                 out=fname,
                 context='sdss', debug=DEBUG))

if os.path.exists(fname): 
    print('Local file:  SUCCESS')
    os.remove(fname)
else:
    print('Local file:  FAILED')


print('######  Output to VOS file')
fname = 'vos://test.id'

if sc.access(fname): 
    os.remove(fname)

print(spec.query(30.0,1.0,0.05,
                 out=fname,
                 context='sdss', debug=DEBUG))

if sc.access(fname): 
    print('VOS file:  SUCCESS')
    sc.rm(fname)
else:
    print('VOS file:  FAILED')

### Single-Spectrum client

In [ ]:
def getSpec (release, run2d, plate, mjd, fiber, bands):
    '''Utility to get a spectrum from the NPY save files. 
    '''
    url = svc_base + "/coadd?release=%s&run2d=%d&plate=%d&mjd=%d&fiber=%d&bands=%s" % \
        (release, run2d, plate, mjd, fiber, bands)
    try:
        r = requests.get (url, timeout=2)
    except Exception as e:
        raise Exception("Error getting data: " + str(e))
    else:
        return np.load(BytesIO(r.content), allow_pickle=False)

In [ ]:
%%time

npy_data = getSpec(release,run2d,plate,mjd,fiber,bands)
if npy_data is None:
    print ('Error: None data')
else:
    print ('Data:  len = ' + str(len(npy_data)) + '  shape: ' + str(npy_data.shape))
    print ('Names: ' + str(npy_data.dtype.names))
    #print (npy_data)

### Specutils

In [ ]:
def to_Spectrum1D (npy_data):
    ''' Convert a Numpy spectrum array to a Spectrum1D object.
    '''
    lamb = 10**npy_data['loglam'] * u.AA 
    flux = npy_data['flux'] * 10**-17 * u.Unit('erg cm-2 s-1 AA-1')

    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
spec1d = to_Spectrum1D(npy_data)

### Pandas Data Frame

In [ ]:
def to_pandas (npy_data):
    return pd.DataFrame(data=npy_data,columns=npy_data.dtype.names)

In [ ]:
df = to_pandas(npy_data)

### Plot Utilities

In [ ]:
def plotSpec (spec):
    '''
    '''
    f, ax = plt.subplots() 
    if isinstance (spec, np.ndarray):
        ax.step(10**df['loglam'], df['flux']) 
    elif isinstance (spec, pd.DataFrame):
        ax.step(10**df['loglam'], df['flux']) 
    elif isinstance (spec, Spectrum1D):
        ax.step(spec.spectral_axis, spec.flux)
    else:
        raise Exception('Unknown format: ' + str(type(spec)))

In [ ]:
plotSpec (npy_data)     # Numpy array
plotSpec (spec1d)       # Spectrum1D
plotSpec (df)

In [ ]:
import numpy as np
x = np.linspace(0, 10, 100)

fig = plt.figure()
plt.plot(x, np.sin(x), '-')
plt.plot(x, np.cos(x), '--');
plt.plot(x, np.sin(x)+np.cos(x), '+');